In [10]:
import mediapipe as mp
import numpy as np
import pandas as pd
import cv2
from typing import Tuple, Union
import math
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

In [7]:
MARGIN = 10  # pixels
ROW_SIZE = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
TEXT_COLOR = (255, 0, 0)  # red

def _normalized_to_pixel_coordinates(
    normalized_x: float, normalized_y: float, image_width: int,
    image_height: int) -> Union[None, Tuple[int, int]]:
  """Converts normalized value pair to pixel coordinates."""

  # Checks if the float value is between 0 and 1.
  def is_valid_normalized_value(value: float) -> bool:
    return (value > 0 or math.isclose(0, value)) and (value < 1 or
                                                      math.isclose(1, value))

  if not (is_valid_normalized_value(normalized_x) and
          is_valid_normalized_value(normalized_y)):
    # TODO: Draw coordinates even if it's outside of the image bounds.
    return None
  x_px = min(math.floor(normalized_x * image_width), image_width - 1)
  y_px = min(math.floor(normalized_y * image_height), image_height - 1)
  return x_px, y_px

def visualize(
    image,
    detection_result
) -> np.ndarray:
  """Draws bounding boxes and keypoints on the input image and return it.
  Args:
    image: The input RGB image.
    detection_result: The list of all "Detection" entities to be visualize.
  Returns:
    Image with bounding boxes.
  """
  annotated_image = image.copy()
  height, width, _ = image.shape

  for detection in detection_result.detections:
    # Draw bounding_box
    bbox = detection.bounding_box
    start_point = bbox.origin_x, bbox.origin_y
    end_point = bbox.origin_x + bbox.width, bbox.origin_y + bbox.height
    cv2.rectangle(annotated_image, start_point, end_point, TEXT_COLOR, 3)

    # Draw keypoints
    for keypoint in detection.keypoints:
      keypoint_px = _normalized_to_pixel_coordinates(keypoint.x, keypoint.y,
                                                     width, height)
      color, thickness, radius = (0, 255, 0), 2, 2
      cv2.circle(annotated_image, keypoint_px, thickness, color, radius)

    # Draw label and score
    category = detection.categories[0]
    category_name = category.category_name
    category_name = '' if category_name is None else category_name
    probability = round(category.score, 2)
    result_text = category_name + ' (' + str(probability) + ')'
    text_location = (MARGIN + bbox.origin_x,
                     MARGIN + ROW_SIZE + bbox.origin_y)
    cv2.putText(annotated_image, result_text, text_location, cv2.FONT_HERSHEY_PLAIN,
                FONT_SIZE, TEXT_COLOR, FONT_THICKNESS)

  return annotated_image

In [2]:
# Use OpenCV’s VideoCapture to start capturing from the webcam.
webcam = cv2.VideoCapture(0)

if not webcam.isOpened():
    print("Could not open webcam")
    exit()

In [3]:
model_path = './models/blaze_face_short_range.tflite'

BaseOptions = mp.tasks.BaseOptions
FaceDetector = mp.tasks.vision.FaceDetector
FaceDetectorOptions = mp.tasks.vision.FaceDetectorOptions
FaceDetectorResult = mp.tasks.vision.FaceDetectorResult
VisionRunningMode = mp.tasks.vision.RunningMode

class storage:
    def __init__(self):
        self.image = np.zeros((480, 640), dtype=np.uint8)

    def get_annotated_image(self):
        return self.image
    
    def save_annotated_image(self, img):
        self.image = img

# 이미지 저장하는 클래스
image_storage = storage()

In [12]:
# Create a face detector instance with the live stream mode:
def print_result(result: FaceDetectorResult, output_image: mp.Image, timestamp_ms: int):
    # print('face detector result: {}'.format(result))
    print(result.detections)
    
    detection_result = result
    mp_image = output_image
    annotated_image = visualize(mp_image.numpy_view(), detection_result)
    image_storage.save_annotated_image(annotated_image)

options = FaceDetectorOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.LIVE_STREAM,
    result_callback=print_result)

In [13]:
with FaceDetector.create_from_options(options) as detector:
    # The detector is initialized. Use it here.

    # Create a loop to read the latest frame from the camera using VideoCapture#read()
    while webcam.isOpened():
        status, frame = webcam.read()
        frame_timestamp_ms = int(webcam.get(cv2.CAP_PROP_POS_MSEC))

        # Convert the frame received from OpenCV to a MediaPipe’s Image object.
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
            
        # Send live image data to perform face detection.
        # The resulqts are accessible via the `result_callback` provided in
        # the `FaceDetectorOptions` object.
        # The face detector must be created with the live stream mode.
        detector.detect_async(mp_image, frame_timestamp_ms)
            
        if status:
            cv2.imshow("test", image_storage.get_annotated_image())
            #cv2.imshow("test", annotated_image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break 

    cv2.destroyAllWindows()

    # Convert the frame received from OpenCV to a MediaPipe’s Image object.

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[Detection(bounding_box=BoundingBox(origin_x=137, origin_y=91, width=184, height=184), categories=[Category(index=0, score=0.6389523148536682, display_name=None, category_name=None)], keypoints=[NormalizedKeypoint(x=0.25618061423301697, y=0.433729887008667, label='', score=0.0), NormalizedKeypoint(x=0.3250783085823059, y=0.3005220293998718, label='', score=0.0), NormalizedKeypoint(x=0.3379981517791748, y=0.4240127205848694, label='', score=0.0), NormalizedKeypoint(x=0.39893120527267456, y=0.45645612478256226, label='', score=0.0), NormalizedKeypoint(x=0.2603752315044403, y=0.5020253658294678, label='', score=0.0), NormalizedKeypoint(x=0.4077780842781067, y=0.21154429018497467, label='', score=0.0)])]
[Detection(bounding_box=BoundingBox(origin_x=76, origin_y=38, width=301, h